<a href="https://colab.research.google.com/github/Yasir323/ML-algorithms-from-scratch/blob/main/Decision_Trees_from_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from typing import List
import math

$Entropy, H(S) = -\sum_i^n p_i log_2(p_i)$

In [2]:
def entropy(class_probabilities: List[float]) -> float:
    """Given a list of class probabilities, compute the entropy"""
    return sum(-p * math.log(p, 2) for p in class_probabilities if p > 0)

In [3]:
entropy([1.0])

0.0

In [4]:
assert entropy([0.5, 0.5]) == 1

In [5]:
from typing import Any
from collections import Counter

In [6]:
def class_probabilities(labels: List[Any]) -> List[float]:
    total_count = len(labels)
    return [count / total_count for count in Counter(labels).values()]

In [7]:
def data_entropy(labels: List[Any]) -> float:
    return entropy(class_probabilities(labels))

In [8]:
print(data_entropy(['a']))
print(data_entropy([True, False]))
print(data_entropy([3, 4, 4, 4]))

0.0
1.0
0.8112781244591328


Partition Entropy, $ H = q_1H(S_1) + ... + q_m H(S_m) $

In [9]:
def partition_entropy(subsets: List[List[Any]]) -> float:
    """Returns the entropy from this partition of data into subsets"""
    total_count = sum(len(subset) for subset in subsets)
    # Return H = H(s1) * q1 + ... + H(Sm) * qm
    return sum(data_entropy(subset) * len(subset) / total_count for subset in subsets)

In [10]:
from typing import NamedTuple, Optional

In [11]:
class Candidate(NamedTuple):
    level: str
    lang: str
    tweets: bool
    phd: bool
    did_well: Optional[bool] = None # allow unlabeld data

In [12]:
inputs = [Candidate('Senior', 'Java', False, False, False),
    Candidate('Senior', 'Java', False, True, False),
    Candidate('Mid', 'Python', False, False, True),
    Candidate('Junior', 'Python', False, False, True),
    Candidate('Junior', 'R', True, False, True),
    Candidate('Junior', 'R', True, True, False),
    Candidate('Mid', 'R', True, True, True),
    Candidate('Senior', 'Python', False, False, False),
    Candidate('Senior', 'R', True, False, True),
    Candidate('Junior', 'Python', True, False, True),
    Candidate('Senior', 'Python', True, True, True),
    Candidate('Mid', 'Python', False, True, True),
    Candidate('Mid', 'Java', True, False, True),
    Candidate('Junior', 'Python', False, True, False)
]

Our tree will consist of decision nodes (which ask a question and direct us
differently depending on the answer) and leaf nodes (which give us a
prediction). We will build it using the relatively simple ID3 algorithm,
which operates in the following manner. Let’s say we’re given some labeled
data, and a list of attributes to consider branching on:

* If the data all have the same label, create a leaf node that predicts
that label and then stop.

* If the list of attributes is empty (i.e., there are no more possible questions to ask), create a leaf node that predicts the most common label and then stop.

* Otherwise, try partitioning the data by each of the attributes.

* Choose the partition with the lowest partition entropy.

* Add a decision node based on the chosen attribute.

* Recur on each partitioned subset using the remaining attributes.

In [13]:
from typing import Dict, TypeVar
from collections import defaultdict

In [14]:
T = TypeVar('T')  # Generic type for inputs

In [15]:
# Function that does the partitioning
def partition_by(inputs: List[T], attribute: str) -> Dict[Any, List[T]]:
    """partition the inputs into lists based on the specified attribute"""
    partitions: Dict[Any, List[T]] = defaultdict(list)
    for input_ in inputs:
        key = getattr(input_, attribute)  # Value of the specified attribute
        partitions[key].append(input_)  # Add input to the correct partition
    return partitions

In [16]:
# Function to compute entropy of a partition
def partition_entropy_by(inputs: List[Any], attribute: str, label_attribute: str) -> float:
    """Compute entropy corresponding to the given partition"""
    # partitions consist of our inputs
    partitions = partition_by(inputs, attribute)
    # But the partition_entropy needs the class labels
    labels = [[getattr(input_,label_attribute) for input_ in partition]
              for partition in partitions.values()]
    return partition_entropy(labels)

In [17]:
for key in ['level', 'lang', 'tweets', 'phd']:
    print(key, partition_entropy_by(inputs, key, 'did_well'))

level 0.6935361388961919
lang 0.8601317128547441
tweets 0.7884504573082896
phd 0.8921589282623617


The lowest entropy comes from splitting on level, so we’ll need to make a
subtree for each possible level value. Every Mid candidate is labeled True,
which means that the Mid subtree is simply a leaf node predicting True. For
Senior candidates, we have a mix of Trues and Falses, so we need to split
again:

In [18]:
senior_inputs = [input_ for input_ in inputs if input_.level == 'Senior']

In [19]:
print(partition_entropy_by(senior_inputs, 'lang', 'did_well'))
print(partition_entropy_by(senior_inputs, 'tweets', 'did_well'))
print(partition_entropy_by(senior_inputs, 'phd', 'did_well') )

0.4
0.0
0.9509775004326938


This shows us that our next split should be on tweets, which results in a
zero-entropy partition. For these Senior-level candidates, “yes” tweets
always result in True while “no” tweets always result in False.

Finally, if we do the same thing for the Junior candidates, we end up
splitting on phd, after which we find that no PhD always results in True and
PhD always results in False.

## Putting it all together
We define a tree to be either:

* a **Leaf** (that predicts a single value), or
* a **Split** (containing an attribute to split on, subtrees for specific
values of that attribute, and possibly a default value to use if we
see an unknown value).

In [20]:
from typing import Union

In [21]:
class Leaf(NamedTuple):
    value: Any

In [22]:
class Split(NamedTuple):
    attribute: str
    subtrees: dict
    default_value: Any = None

In [23]:
DecisionTree = Union[Leaf, Split]

In [24]:
# With this representation, our hiring tree would look like:
hiring_tree = Split('level', { # first, consider "level"
    'Junior': Split('phd', { # if level is "Junior", next look at "phd"
        False: Leaf(True), # if "phd" is False, predict True
        True: Leaf(False) # if "phd" is True, predict False
    }),
    'Mid': Leaf(True), # if level is "Mid", just predict True
    'Senior': Split('tweets', { # if level is "Senior", look at "tweets"
        False: Leaf(False), # if "tweets" is False, predict False
        True: Leaf(True) # if "tweets" is True, predict True
    })
})

There’s still the question of what to do if we encounter an unexpected (or
missing) attribute value. What should our hiring tree do if it encounters a
candidate whose level is Intern? We’ll handle this case by populating the
default_value attribute with the most common label.

In [25]:
def classify(tree: DecisionTree, input_: Any) -> Any:
    """Classify the input using the given decision tree"""
    # If this is a leaf node, return its value
    if isinstance(tree, Leaf):
        return tree.value
    # Otherwise this tree consists of an an attribute to split on
    # and a dictionary whose keys are values of that attribute
    # and whose values are subtrees to consider next
    subtree_key = getattr(input_, tree.attribute)

    # If no subtree for key
    if subtree_key not in tree.subtrees:
        # Return the default value
        return tree.default_value

    # Choose the appropriate subtree
    subtree = tree.subtrees[subtree_key]
    # And use it to classify the input
    return classify(subtree, input_)

## Build the tree representation

In [26]:
def build_tree_id3(inputs: List[Any],
                   split_attributes: List[str],
                   target_attribute: str) -> DecisionTree:
    
    # Count Target labels
    label_counts = Counter(getattr(input_, target_attribute)
                            for input_ in inputs)
    most_common_label = label_counts.most_common(1)[0][0]

    # If there's a unique label, predict that only
    if len(label_counts) == 1:
        return Leaf(most_common_label)

    # If no split attribtes left, return the , majority label
    if not split_attributes:
        return Leaf(most_common_label)

    # Otherwise split by the best attribute

    def split_entropy(attribute: str) -> float:
        """Helper funtion for finding the best attribute"""
        return partition_entropy_by(inputs, attribute, target_attribute)

    best_attribute = min(split_attributes, key=split_entropy)

    partitions = partition_by(inputs, best_attribute)
    new_attributes = [a for a in split_attributes if a != best_attribute]

    # Recursively build subtrees
    subtrees = {
        attribute_value: build_tree_id3(
            subset,
            new_attributes,
            target_attribute
        ) for attribute_value, subset in partitions.items()
    }
    return Split(best_attribute, subtrees, default_value=most_common_label)

In [27]:
tree = build_tree_id3(
    inputs,
    ['level', 'lang', 'tweets', 'phd'],
    'did_well'
)

In [28]:
# Should be True
classify(tree, Candidate("Junior", "Java", True, False))

True

In [29]:
# Should be True
classify(tree, Candidate("Junior", "Java", True, True))

False

In [30]:
# And also to data with unexpected values:
# Should predict True
classify(tree, Candidate("Intern", "Java", True, True))

True